In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')

import re
import os

In [2]:
! ls

 CARBON_BED_molecular_35A.data	      displaced_carbon_bed_VMD.data
 carbon_bed_OPLS-AA_QeD_charges.mol  'from VMD data file to LAMMPS mol.ipynb'
 carbon_bed_OPSLAA_PARMS.lammps


In [3]:
with open("displaced_carbon_bed_VMD.data", "r") as file:
    file_content = file.readlines()
# Create a DataFrame with each line as a separate row in a single column
data = pd.DataFrame(file_content, columns=['data']) 

In [4]:
slices = {
    "coords":(43,2742),        # 2700
    "bonds":(5449,9333),       # 3885
    "angles":(9337,16786),     # 7450
    "dihedrals":(16790,31144), # 14355
    "impropers":(31148,33527)  # 2380
}

In [5]:
def get_coords(slices):
    coords = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    coords["data"] = coords["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    coords["atom id"] = coords["data"].apply(lambda x: x[0]).astype(int)
    coords["molecule id"] = coords["data"].apply(lambda x: x[1]).astype(int)
    coords["atom type"] = coords["data"].apply(lambda x: x[2]).astype(int)
    coords["VMD charge"] = coords["data"].apply(lambda x: x[3]).astype(float)
    coords["x"] = coords["data"].apply(lambda x: x[4]).astype(float)
    coords["y"] = coords["data"].apply(lambda x: x[5]).astype(float)
    coords["z"] = coords["data"].apply(lambda x: x[6]).astype(float)
    return coords

coords = get_coords(slices["coords"])

def get_bonds(slices):
    bonds = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    bonds["data"] = bonds["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    bonds["bond id"] = bonds["data"].apply(lambda x: x[0]).astype(int)
    bonds["bond type"] = bonds["data"].apply(lambda x: x[1]).astype(int)
    bonds["atom1"] = bonds["data"].apply(lambda x: x[2]).astype(int)
    bonds["atom2"] = bonds["data"].apply(lambda x: x[3]).astype(int)
    return bonds

bonds = get_bonds(slices["bonds"])

def get_angles(slices):
    angles = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    angles["data"] = angles["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    angles["angle id"] = angles["data"].apply(lambda x: x[0]).astype(int)
    angles["angle type"] = angles["data"].apply(lambda x: x[1]).astype(int)
    angles["atom1"] = angles["data"].apply(lambda x: x[2]).astype(int)
    angles["atom2"] = angles["data"].apply(lambda x: x[3]).astype(int)
    angles["atom3"] = angles["data"].apply(lambda x: x[4]).astype(int)
    return angles

angles = get_angles(slices["angles"])

def get_dihedrals(slices):
    dihedrals = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    dihedrals["data"] = dihedrals["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    dihedrals["dihedral id"] = dihedrals["data"].apply(lambda x: x[0]).astype(int)
    dihedrals["dihedral type"] = dihedrals["data"].apply(lambda x: x[1]).astype(int)
    dihedrals["atom1"] = dihedrals["data"].apply(lambda x: x[2]).astype(int)
    dihedrals["atom2"] = dihedrals["data"].apply(lambda x: x[3]).astype(int)
    dihedrals["atom3"] = dihedrals["data"].apply(lambda x: x[4]).astype(int)
    dihedrals["atom4"] = dihedrals["data"].apply(lambda x: x[5]).astype(int)
    return dihedrals

dihedrals = get_dihedrals(slices["dihedrals"])

def get_impropers(slices):
    impropers = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    impropers["data"] = impropers["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    impropers["improper id"] = impropers["data"].apply(lambda x: x[0]).astype(int)
    impropers["improper type"] = impropers["data"].apply(lambda x: x[1]).astype(int)
    impropers["atom1"] = impropers["data"].apply(lambda x: x[2]).astype(int)
    impropers["atom2"] = impropers["data"].apply(lambda x: x[3]).astype(int)
    impropers["atom3"] = impropers["data"].apply(lambda x: x[4]).astype(int)
    impropers["atom4"] = impropers["data"].apply(lambda x: x[5]).astype(int)
    return impropers

impropers = get_impropers(slices["impropers"])

# FIX CHARGES

### REMARKS

* VMD dataset has its simulation cell as -20 20, -20 20, 0 100.
* MATSQ dataset has its simulation cell as 0 40, 0 40, 0 100.
* Only modify location of MATSQ dataset.

In [6]:
def get_MATSQ_coords(slices):
    with open("../MATSQ/carbon_bed_structure_dreiding.lammps", "r") as file:
        file_content = file.readlines()
    # Create a DataFrame with each line as a separate row in a single column
    MATSQ = pd.DataFrame(file_content, columns=['data'])     
    
    coords = MATSQ.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    coords["data"] = coords["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    coords["atom id"] = coords["data"].apply(lambda x: x[0]).astype(int)
    coords["molecule id"] = coords["data"].apply(lambda x: x[1]).astype(int)
    coords["atom type"] = coords["data"].apply(lambda x: x[2]).astype(int)
    coords["charge"] = coords["data"].apply(lambda x: x[3]).astype(float)
    coords["x"] = coords["data"].apply(lambda x: x[4]).astype(float)
    coords["x"] = coords["x"] - 20 
    coords["y"] = coords["data"].apply(lambda x: x[5]).astype(float)
    coords["y"] = coords["y"] - 20
    coords["z"] = coords["data"].apply(lambda x: x[6]).astype(float)
    return coords

MATSQ_coords = get_MATSQ_coords([25,2724])

In [7]:
MATSQ_coords["loc_key"] = MATSQ_coords["x"].apply(lambda x: round(x,3)).astype(str) + \
                          MATSQ_coords["y"].apply(lambda x: round(x,3)).astype(str) + \
                          MATSQ_coords["z"].apply(lambda x: round(x,3)).astype(str)
MATSQ_coords["sum_key"] = MATSQ_coords["x"] + MATSQ_coords["y"] + MATSQ_coords["z"]
MATSQ_coords["sum_key"] = MATSQ_coords["sum_key"].apply(lambda x: round(x,3))
MATSQ_coords["sum_key"] = MATSQ_coords["sum_key"].apply(lambda x: x + 0.001 if x > 0 else x - 0.001) 
MATSQ_coords["sum_key"] = MATSQ_coords["sum_key"].apply(lambda x: round(x,3))

In [8]:
dict_charges_loc = dict(zip(MATSQ_coords["loc_key"].values,MATSQ_coords["charge"].values))
dict_charges_sum = dict(zip(MATSQ_coords["sum_key"].values,MATSQ_coords["charge"].values))

In [9]:
coords["loc_key"] = coords["x"].apply(lambda x: round(x,3)).astype(str) + \
                    coords["y"].apply(lambda x: round(x,3)).astype(str) + \
                    coords["z"].apply(lambda x: round(x,3)).astype(str)
coords["sum_key"] = coords["x"] + coords["y"] + coords["z"]
coords["sum_key"] = coords["sum_key"].apply(lambda x: round(x,3))

#### First, mapping by location

In [10]:
coords["charge"] = coords["loc_key"].map(dict_charges_loc)
coords.loc[coords[coords["charge"].isna()]["sum_key"].map(dict_charges_sum).index,
           "charge"] = coords[coords["charge"].isna()]["sum_key"].map(dict_charges_sum).values

In [11]:
coords[coords["sum_key"].isin([5.265])]
#coords[coords["charge"].isna()]["sum_key"].unique()

,data,atom id,molecule id,atom type,VMD charge,x,y,z,loc_key,sum_key,charge
1782,"[1649, 1649, 1, 0, 12.407499999999999, -17.192...",1649,1649,1,0.0,12.4075,-17.192337,10.05,12.407-17.19210.05,5.265,NaN


In [12]:
MATSQ_coords[MATSQ_coords["sum_key"].isin([5.267])]

,data,atom id,molecule id,atom type,charge,x,y,z,loc_key,sum_key
1648,"[1649, 4, 1, -0.05957, 32.40800000, 2.80800000...",1649,4,1,-0.05957,12.408,-17.192,10.05,12.408-17.19210.05,5.267


### Lol, they seem to conserve the atom ID :v qude'e como est'upida. 

In [13]:
dict_charges_ID = dict(zip(MATSQ_coords["atom id"].values,MATSQ_coords["charge"].values))

In [14]:
coords.loc[coords[coords["charge"].isna()]["atom id"].map(dict_charges_ID).index,
           "charge"] = coords[coords["charge"].isna()]["atom id"].map(dict_charges_ID).values

In [15]:
pd.Series(coords.sort_values("atom id").sort_values("charge")["atom id"].values == 
          MATSQ_coords.sort_values("atom id").sort_values("charge")["atom id"].values).unique()

array([ True])

In [16]:
coords["charge"].isna().unique()

array([False])

In [17]:
PARAMS_coeffs = {"atom type":6, "bonds":6, "angles":4, "dihedrals":16, "impropers":1}

In [18]:
def write_mol_file():
    f = open("carbon_bed_OPLS-AA_QeD_charges.mol", "w")
    f.write("# Created by Karina Chiñas Fuentes | Python | 16.08.24\n\n")
    f.write(f"\n{len(coords)} atoms\n{len(bonds)} bonds\n{len(angles)} angles\n{len(dihedrals)} dihedrals"+
            f"\n{len(impropers)} impropers")
    f.write("\n\n")
    f.write("Coords\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['x'][i]}\t{coords['y'][i]}\t{coords['z'][i]}\n")
    f.write("\nTypes\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]} {PARAMS_coeffs['atom type']}\n")
    f.write("\nCharges\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['charge'][i]}\n")
    f.write("\nBonds\n\n")
    for i in range(len(bonds)):
        f.write(f"{bonds['bond id'][i]}\t{PARAMS_coeffs['bonds']}\t"+
                f"{bonds['atom1'][i]}\t{bonds['atom2'][i]}\n")
    f.write("\nAngles\n\n")
    for i in range(len(angles)):
        f.write(f"{angles['angle id'][i]}\t{PARAMS_coeffs['angles']}\t"+
                f"{angles['atom1'][i]}\t{angles['atom2'][i]}\t{angles['atom3'][i]}\n")
    f.write("\nDihedrals\n\n")
    for i in range(len(dihedrals)):
        f.write(f"{dihedrals['dihedral id'][i]}\t{PARAMS_coeffs['dihedrals']}\t{dihedrals['atom1'][i]}\t"+
                f"{dihedrals['atom2'][i]}\t{dihedrals['atom3'][i]}\t{dihedrals['atom4'][i]}\n")
    f.write("\nImpropers\n\n")
    for i in range(len(impropers)):
        f.write(f"{impropers['improper id'][i]}\t{PARAMS_coeffs['impropers']}\t{impropers['atom1'][i]}\t"+
                f"{impropers['atom2'][i]}\t{impropers['atom3'][i]}\t{impropers['atom4'][i]}\n")

In [19]:
write_mol_file()

In [20]:
! ls

 CARBON_BED_molecular_35A.data	      displaced_carbon_bed_VMD.data
 carbon_bed_OPLS-AA_QeD_charges.mol  'from VMD data file to LAMMPS mol.ipynb'
 carbon_bed_OPSLAA_PARMS.lammps


In [21]:
#! cp carbon_bed_OPLS-AA_QeD_charges.mol /home/kchinas/Documents/PhD/LAMMPS/CL/2ndMODEL/1stAttmp/merge/VMD_C/data

In [25]:
pd.options.display.float_format = "{:.4f}".format

coords["x"] = coords["x"].apply(lambda x: round(x,4))
coords["y"] = coords["y"].apply(lambda x: round(x,4))
coords["z"] = coords["z"].apply(lambda x: round(x,4))

In [26]:
coords

,data,atom id,molecule id,atom type,VMD charge,x,y,z,loc_key,sum_key,charge
0,"[1, 1, 1, 0, -18.0795, -18.420361, 0, 0, 0, 0]",1,1,1,0.0000,-18.0795,-18.4204,0.0000,-18.079-18.420.0,-36.5000,-0.0170
1,"[2, 2, 1, 0, -18.7885, -17.192337000000002, 0,...",2,2,1,0.0000,-18.7885,-17.1923,0.0000,-18.788-17.1920.0,-35.9810,-0.0442
2,"[3, 3, 1, 0, -16.6615, -18.420361, 0, 0, 0, 0]",3,3,1,0.0000,-16.6615,-18.4204,0.0000,-16.662-18.420.0,-35.0820,-0.0514
3,"[4, 4, 1, 0, -15.9525, -17.192337000000002, 0,...",4,4,1,0.0000,-15.9525,-17.1923,0.0000,-15.953-17.1920.0,-33.1450,0.0516
4,"[5, 5, 1, 0, -13.8255, -18.420361, 0, 0, 0, 0]",5,5,1,0.0000,-13.8255,-18.4204,0.0000,-13.825-18.420.0,-32.2460,-0.0596
...,...,...,...,...,...,...,...,...,...,...,...
2695,"[2696, 2696, 1, 0, 13.825500999999996, 17.1923...",2696,2696,1,0.0000,13.8255,17.1923,13.4000,13.82617.19213.4,44.4180,-0.0596
2696,"[2697, 2697, 1, 0, 15.952500999999998, 15.9643...",2697,2697,1,0.0000,15.9525,15.9643,13.4000,15.95315.96413.4,45.3170,0.0451
2697,"[2698, 2698, 1, 0, 15.243502, 17.1923370000000...",2698,2698,1,0.0000,15.2435,17.1923,13.4000,15.24417.19213.4,45.8360,-0.0599
2698,"[2699, 2699, 1, 0, 17.370500999999997, 15.9643...",2699,2699,1,0.0000,17.3705,15.9643,13.4000,17.37115.96413.4,46.7350,0.0208


# Write LAMMPS data file

In [61]:
# Fix for considering other atoms:
PARAMS_C = {"atom type":6, "bonds":6, "angles":4, "dihedrals":16, "impropers":1}
totTypes = {"atom":8, "bond":7, "angle":5, "dihedral":16, "improper":1}

In [68]:
def write_lmpdat_file():
    f = open("carbon_bed_OPLS-AA_QeD_charges.data", "w")
    f.write("# Created by Karina Chiñas Fuentes | Python | 19.08.24\n\n")
    f.write(f"{len(coords)} atoms\n{len(bonds)} bonds\n{len(angles)} angles\n"+
            f"{len(dihedrals)} dihedrals\n{len(impropers)} impropers\n"+
            f"{total_amount_of_types['atom']} atom types\n{total_amount_of_types['bond']} bond types\n"+
            f"{total_amount_of_types['angle']} angle types\n{total_amount_of_types['dihedral']} dihedral types"+
            f"\n{total_amount_of_types['improper']} improper types\n\n")
    # simulation box dimensions
    f.write("-20 20 xlo xhi\n")
    f.write("-20 20 ylo yhi\n")
    f.write(" 0 300 zlo zhi\n\n")
    f.write(f"# Pair Coeffs\n# {PARAMS_C['atom type']}  CA\n\n# Bond Coeffs\n# "+
            f"{PARAMS_C['bonds']}  CA-CA\n\n# Angle Coeffs\n# {PARAMS_C['angles']}  CA-CA-CA\n\n"+
            f"# Dihedral Coeffs\n# {PARAMS_C['dihedrals']}  CA-CA-CA-CA\n\n# Improper Coeffs\n#"+
            f" {PARAMS_C['impropers']}  CA-CA-CA-CA\n\n")
    # masses
    f.write("Masses\n\n")
    f.write("1     18.998400  # F_  | Nafion   (DREIDING)\n"+
            "2     12.011000  # C_3 | Nafion   (DREIDING)\n"+
            "3     15.999400  # O_3 | Nafion   (DREIDING)\n"+
            "4     15.999400  # O_2 | Nafion   (DREIDING)\n"+
            "5     32.060000  # S_3 | Nafion   (DREIDING)\n"+
            "6     12.011000  # C   | Graphite (OPLSAA-AA)\n"+
            "7     1.008      # H   | Water    (SPC/Fw)\n"+
            "8     15.999400  # O   | Water    (SPC/Fw)\n\n")
    # atoms
    f.write("Atoms\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['atom id'][i]}\t{PARAMS_C['atom type']}\t"+
                f"{coords['charge'][i]}\t{coords['x'][i]}\t{coords['y'][i]}\t{coords['z'][i]} # CA GRA\n")

In [69]:
write_lmpdat_file()

In [70]:
! code carbon_bed_OPLS-AA_QeD_charges.data